In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna
import dagshub
import pickle


dagshub.init(repo_owner='satyajeetrai007', repo_name='Youtube-Comment-Sentiment-Analysis', mlflow=True)
mlflow.set_experiment("Algorithms with HyperParameter tuning")

df = pd.read_csv('data_preprocessed.csv').dropna().drop_duplicates()
df.shape

Initialized MLflow to track repo "satyajeetrai007/Youtube-Comment-Sentiment-Analysis"

Repository satyajeetrai007/Youtube-Comment-Sentiment-Analysis initialized!

(36243, 2)

In [6]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for KNN

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 2000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model

        with open(f"{model_name}_model", "wb") as f:
            pickle.dump(model, f)

        with open(f"{model_name}_model", "rb") as f:
            clf = pickle.load(f)

        mlflow.log_artifact(f"{model_name}_model", "model")
        


# Step 6: Optuna objective function for KNN
def objective_knn(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 3, 30)  # Tuning the number of neighbors
    p = trial.suggest_categorical('p', [1, 2])  # Tuning the distance metric (1 for Manhattan, 2 for Euclidean)

    # KNeighborsClassifier setup
    model = KNeighborsClassifier(n_neighbors=n_neighbors, p=p)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for KNN, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_knn, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'], p=best_params['p'])

    # Log the best model with MLflow, passing the algo_name as "KNN"
    log_mlflow("KNN", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for KNN
run_optuna_experiment()

[I 2025-08-26 20:54:54,472] A new study created in memory with name: no-name-50e8a5a9-dfd2-4509-8c35-bc9682bae600
[I 2025-08-26 20:55:07,027] Trial 0 finished with value: 0.36606382978723406 and parameters: {'n_neighbors': 10, 'p': 1}. Best is trial 0 with value: 0.36606382978723406.
[I 2025-08-26 20:55:11,431] Trial 1 finished with value: 0.36021276595744683 and parameters: {'n_neighbors': 15, 'p': 1}. Best is trial 0 with value: 0.36606382978723406.
[I 2025-08-26 20:55:18,497] Trial 2 finished with value: 0.395531914893617 and parameters: {'n_neighbors': 3, 'p': 1}. Best is trial 2 with value: 0.395531914893617.
[I 2025-08-26 20:55:24,930] Trial 3 finished with value: 0.3791489361702128 and parameters: {'n_neighbors': 5, 'p': 1}. Best is trial 2 with value: 0.395531914893617.
[I 2025-08-26 20:55:29,171] Trial 4 finished with value: 0.3537234042553192 and parameters: {'n_neighbors': 21, 'p': 1}. Best is trial 2 with value: 0.395531914893617.
[I 2025-08-26 20:55:33,379] Trial 5 finishe

🏃 View run KNN_SMOTE_TFIDF_Trigrams at: https://dagshub.com/satyajeetrai007/Youtube-Comment-Sentiment-Analysis.mlflow/#/experiments/6/runs/1a7028ae192148ad97948f3e36283eac
🧪 View experiment at: https://dagshub.com/satyajeetrai007/Youtube-Comment-Sentiment-Analysis.mlflow/#/experiments/6
